In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
y = data.DEATH_EVENT.values
x_data = data.drop(["DEATH_EVENT"], axis = 1)
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data))
x

# TRAIN/TEST SPLIT

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

print("x_train shape: {}\nx_test shape: {}\ny_train shape: {}\ny_test shape: {}".format(x_train.shape, x_test.shape, y_train.shape, y_test.shape))

x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

print("\nx_train shape: {}\nx_test shape: {}\ny_train shape: {}\ny_test shape: {}".format(x_train.shape, x_test.shape, y_train.shape, y_test.shape))

In [ ]:
def initialize_weight_and_bias(dimension):
    
    w = np.full((dimension,1),0.01)
    b = 0.0
    
    return w,b

def sigmoid(z):
    
    y_head = 1 / (1 + np.exp(-z))
    
    return y_head

In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    
    # forward propagation
    
    z = (np.dot(w.T,x_train) + b)
    y_head = sigmoid(z)
    loss = -(y_train*np.log(y_head)) - (1-y_train)*np.log(1-y_head)
    cost = np.sum(loss) / x_train.shape[1]
    
    # backward propagation
    
    derivative_weight = (np.dot(x_train,((y_head - y_train).T))) / x_train.shape[1]
    derivative_bias = (np.sum(y_head - y_train)) / x_train.shape[1]
    
    gradients = {"derivative_weight": derivative_weight, "derivative_bias": derivative_bias}
    
    return cost,gradients

In [ ]:
def update(w,b,x_train,y_train,learning_rate, num_of_iterations):
    
    cost_list = []
    cost_list2 = []
    index = []
    
    for i in range(num_of_iterations):
        
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        
        if i %10 == 0:
            
            cost_list2.append(cost)
            index.append(i)
            print("Cost after iteration %i : %f" %(i,cost))
        
    parameters = {"weight": w, "bias": b}
    plt.figure(figsize=[10,10])
    plt.plot(index,cost_list2, color = "red", linewidth = 3)
    plt.xticks(index,rotation = "vertical")
    plt.xlabel("Number of Iteration")
    plt.ylabel("Cost")
    plt.show()
    
    return parameters,gradients,cost_list

In [ ]:
def prediction(w,b,x_test):
    
    z = sigmoid(np.dot(w.T,x_test) + b)
    y_prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(z.shape[1]):
        
        if z[0,i] <= 0.5:
            
            y_prediction[0,i] = 0
            
        else:
            
            y_prediction[0,i] = 1
            
    return y_prediction

In [ ]:
def logistic_regression(x_train,y_train,x_test,y_test,learning_rate,num_iterations):
    
    dimension = x_train.shape[0]
    w,b = initialize_weight_and_bias(dimension)
    parameters,gradients,cost_list = update(w,b,x_train,y_train,learning_rate,num_iterations)
    
    
    y_prediction_test = prediction(parameters["weight"], parameters["bias"], x_test)
    
    print("Test Accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test))*100))
    

logistic_regression(x_train,y_train,x_test,y_test,learning_rate = 3, num_iterations = 300)